In [1]:
import pandas as pd
import numpy as np
import os
import FinanceDataReader as fdr
import pickle
import lightgbm

from tqdm import tqdm

In [2]:
stock_list = pd.read_csv('data/stock_list.csv')
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))

In [3]:
start_date = '20210104'
end_date = '20211126'

In [4]:
sample_code = stock_list.loc[0,'종목코드']

sample = fdr.DataReader(sample_code, start = start_date, end = end_date).reset_index()
sample.head()

,Date,Open,High,Low,Close,Volume,Change
0,2021-01-04,81000,84400,80200,83000,38655276,0.024691
1,2021-01-05,81600,83900,81600,83900,35335669,0.010843
2,2021-01-06,83300,84500,82100,82200,42089013,-0.020262
3,2021-01-07,82800,84200,82700,82900,32644642,0.008516
4,2021-01-08,83300,90000,83000,88800,59013307,0.071170


# KOSPI, KOSDAQ, KS200, KQ150, 나스닥, 다우, 원달러 환율, 비트코인 가격 데이터 가져오기

In [5]:
index_dict = dict()

try:
    # KS11 (KOSPI 지수)
    index_dict['KS11'] = fdr.DataReader('KS11', start = start_date, end = end_date).reset_index()
except:
    print('KOSPI 없음')

try:
    # 코스닥 지수
    index_dict['KQ11'] = fdr.DataReader('KQ11', start = start_date, end = end_date).reset_index()
except:
    print('코스닥 없음')

try:
    # KS200 (KOSPI 지수)
    index_dict['KS200'] = fdr.DataReader('KS200', start = start_date, end = end_date).reset_index()
except:
    print('KS200 없음')

try:
    # KQ150 코스닥 지수
    index_dict['KQ150'] = fdr.DataReader('KQ150', start = start_date, end = end_date).reset_index()
except:
    print('KQ150 코스닥 없음')

try:
    # 나스닥지수
    index_dict['IXIC'] = fdr.DataReader('IXIC', start = start_date, end = end_date).reset_index()
except:
    print('나스닥 없음')

try:
    # 다우지수
    index_dict['DJI'] = fdr.DataReader('DJI', start = start_date, end = end_date).reset_index()
except:
    print('다우 없음')

try:
    # 원달러 환율
    index_dict['USD/KRW'] = fdr.DataReader('USD/KRW', start = start_date, end = end_date).reset_index()
except:
    print('달러 없음')

try:
    # 비트코인 원화 가격 (빗썸)
    index_dict['BTC/KRW'] = fdr.DataReader('BTC/KRW', start = start_date, end = end_date).reset_index()
except:
    print('코인 없음')

## 추가

try:
    # 1년만기 한국국채 수익률
    index_dict['KR1YT=RR'] = fdr.DataReader('KR1YT=RR', start = start_date, end = end_date).reset_index()
except:
    print('1년 국채 없음')

try:
    # 10년만기 한국국채 수익률
    index_dict['KR10YT=RR'] = fdr.DataReader('KR10YT=RR', start = start_date, end = end_date).reset_index()
except:
    print('10년 국채 없음 없음')


In [6]:
for k in index_dict.keys():
    rename = []
    for name in index_dict[k].columns:
        rename.append(f'{k}_{name}')
    temp = index_dict[k]
    temp.columns = ['Date'] + rename[1:]
    index_dict[k] = temp

In [7]:
index_dict['USD/KRW'].head()

,Date,USD/KRW_Close,USD/KRW_Open,USD/KRW_High,USD/KRW_Low,USD/KRW_Change
0,2021-01-04,1086.48,1085.73,1088.30,1080.02,0.0016
1,2021-01-05,1086.42,1086.69,1090.33,1082.04,-0.0001
2,2021-01-06,1087.93,1087.40,1089.79,1083.91,0.0014
3,2021-01-07,1094.28,1088.03,1096.78,1085.42,0.0058
4,2021-01-08,1092.93,1094.35,1099.21,1088.79,-0.0012


# 코로나 데이터 활용
## 출처: http://ncov.mohw.go.kr/

- 계(명) : covid_total
- 국내발생(명) : covid_domestic
- 해외유입(명) : covid_foreign_import
- 사망(명) : covid_dead

In [8]:
covid = pd.read_csv('data/코로나바이러스감염증-19_확진환자_발생현황_211126.csv')
covid = covid[-321:].reset_index(drop=True)
covid.columns = ['Date','covid_total', 'covid_domestic', 'covid_foreign_import', 'covid_dead']
covid = covid.replace(',', '', regex=True)
covid.Date = covid.Date.map(lambda x:pd.Timestamp(x))
covid.covid_total = covid.covid_total.astype(np.int64)
covid.covid_domestic = covid.covid_domestic.astype(np.int64)
covid.covid_foreign_import = covid.covid_foreign_import.astype(np.int64)
covid.covid_dead = covid.covid_dead.astype(np.int64)
type(covid.covid_dead[0])

numpy.int64

In [9]:
def NMAE(true, pred):
    score = np.mean(np.abs(true-pred) / true)
    return score

### 이전 가격 추가하는 부분은 2021 농산물 가격예측 AI 경진대회 베이스라인을 참고 하였습니다.

### feature 추가
#### 기본 feature : 'Open', 'High', 'Low', 'Close', 'Volume', 'Change' 6개에서 57개 추가 총 63개
- High - Low
- Close - Open
- Close * Volume
- Open * Volume
- High * Volume
- Low * Volume
- 지수 : 8 * 6 - 1 = 47개 (* USD/KRW는 Volume값이 없습니다.)
- 코로나 : 4개

### use_features에 포함된 feature는 1~100일 전 데이터를 feature로 추가.

In [10]:
def feature_engineering(df, days = 100) :
    use_features = ['Open', 'High', 'Low', 'Close', 'Volume', 'Change', 
                    'covid_total', 'covid_domestic', 'covid_foreign_import', 
                    'covid_dead',
                    'c_v', 'o_v', 'h_v', 'l_v',
                    'KS11_Open', 'KQ11_Open', 'KS200_Open', 'KQ150_Open', 'USD/KRW_Open',
                    'KS11_High', 'KQ11_High', 'USD/KRW_High', 'KS200_High', 'KQ150_High',
                    'KS11_Low', 'KQ11_Low', 'USD/KRW_Low', 'KS200_Low', 'KQ150_Low',
                    'USD/KRW_Close', 'KS11_Close', 'KQ11_Close',  'KS200_Close', 'KQ150_Close',
                    'KS11_Volume', 'KQ11_Volume', 'KS200_Volume', 'KQ150_Volume', #'USD/KRW_Volume', 볼륨이 없는 달러 
                    'USD/KRW_Change', 'KS11_Change', 'KQ11_Change', 'KS200_Change', 'KQ150_Change'
                    ]
    important_indexs = ['KS11', 'KQ11', 'KS200', 'KQ150', 'USD/KRW', 'IXIC','DJI','BTC/KRW', 'KR1YT=RR', 'KR10YT=RR']

    df = pd.merge(df, covid[['Date',
                            'covid_total', 
                            'covid_domestic', 
                            'covid_foreign_import', 
                            'covid_dead'
                            ]], on='Date') 
    
    for index in important_indexs:
      df = pd.merge(df, index_dict[index], on='Date', how='left')

    df = df.ffill()

    df['high-low'] = df['High'] - df['Low']
    df['close-open'] = df['Close'] - df['Open']

    df['c_v'] = df['Close'] * df['Volume']
    df['o_v'] = df['Open'] * df['Volume']
    df['h_v'] = df['High'] * df['Volume']
    df['l_v'] = df['Low'] * df['Volume'] 


    # use_features * days * row
    for col in use_features:
      for lag in range(1,days+1) :
        df[f'{col}_day_{lag}'] = -1
        for index in range(lag, len(df)) :
          df.loc[index, f'{col}_day_{lag}'] = df[col][index-lag] # 1~days전 가격을 feature로 추가
    

    for day in ['after_1', 'after_2', 'after_3', 'after_4', 'after_5'] :
      df[day] = 0
      n_day = int(day[-1])
      for index in range(len(df)) :
        try : df[day][index] = df['Close'][index + n_day]
        except : continue
    
    return df

In [11]:
test = feature_engineering(sample)

/tmp/ipykernel_1457/1446661267.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_day_{lag}'] = -1
/tmp/ipykernel_1457/1446661267.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[day] = 0
/tmp/ipykernel_1457/1446661267.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  try : df[day][index] = df['Close'][index + n_day]


In [12]:
test.head()

,Date,Open,High,Low,Close,Volume,Change,KS11_KS11_Close,KS11_KS11_Open,KS11_KS11_High,...,KQ150_Change_day_96,KQ150_Change_day_97,KQ150_Change_day_98,KQ150_Change_day_99,KQ150_Change_day_100,after_1,after_2,after_3,after_4,after_5
0,2021-01-11,90000,96800,89500,91000,90306177,0.024775,3148.45,3161.90,3266.23,...,-1.0,-1.0,-1.0,-1.0,-1.0,90600,89700,89700,88000,85000
1,2021-01-12,90300,91400,87800,90600,48682416,-0.004396,3125.95,3145.87,3154.79,...,-1.0,-1.0,-1.0,-1.0,-1.0,89700,89700,88000,85000,87000
2,2021-01-13,89800,91200,89100,89700,36068848,-0.009934,3148.29,3128.26,3164.37,...,-1.0,-1.0,-1.0,-1.0,-1.0,89700,88000,85000,87000,87200
3,2021-01-14,88700,90000,88700,89700,26393970,0.000000,3149.93,3148.65,3159.03,...,-1.0,-1.0,-1.0,-1.0,-1.0,88000,85000,87000,87200,88100
4,2021-01-15,89800,91800,88000,88000,33431809,-0.018952,3085.90,3153.84,3189.90,...,-1.0,-1.0,-1.0,-1.0,-1.0,85000,87000,87200,88100,86800


In [13]:
sample_name = 'data/sample_submission.csv'
sample_submission = pd.read_csv(sample_name)

# 11월 1일 ~ 11월 5일 주식 종가 예측

In [14]:
model = lightgbm.LGBMRegressor(random_state=13)

days = 100
label_columns = ['after_1', 'after_2', 'after_3', 'after_4', 'after_5', ]
predict_days = 5 * -1 

accuracy = []
for code in tqdm(stock_list['종목코드'].values):
    data = fdr.DataReader(code, start = start_date, end = end_date).reset_index()
    data = feature_engineering(data)
    data = data[:-15]

    final_x = data.iloc[-1:]
    final_x = final_x.drop(['Date'] + label_columns, axis=1)
    
    x_val = data.iloc[-6:-5]
    x_val = x_val.drop(['Date'] + label_columns, axis=1)
    
    y_val = data.Close[predict_days:]

    # 이전 day 값이 없는 row 제외, 예측하는 날의 row 제외.
    train = data.drop(['Date'], axis=1)[days:predict_days]

    # after 1~5 columns 제외.
    x_train = train.drop(label_columns, axis=1)

    predictions = []
    final_predictions = []
    answers = []
    # 1일 후 부터 5일 후 까지 각각 예측.
    for j in range(1, 6):
        y_train = train[f'after_{j}']
        try:
            model.fit(x_train, y_train)
        except:
            print(f'code:{code}')

        output = model.predict(x_val)
        predictions.append(output[0])

        final_output = model.predict(final_x)
        final_predictions.append(final_output[0])
        
    accuracy.append(NMAE(predictions, y_val))
    sample_submission.loc[:4,code] = predictions 
print(f'na:{sample_submission.isna().sum().sum()}')
print(f'acc:{sum(accuracy)/len(accuracy)}')

  0%|                                                                                                                                                                                                                 | 0/370 [00:00<?, ?it/s]/tmp/ipykernel_1457/1446661267.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_day_{lag}'] = -1
/tmp/ipykernel_1457/1446661267.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[day] = 0
/tmp/ipykernel_1457/1446661267.py:56: SettingWithCopyWarning: 
A value is trying to be

na:0
acc:0.00577362521377838


In [15]:
sample_submission['000060']

0    27888.279695
1    29232.221942
2    30262.127615
3    29487.604504
4    29560.781304
5        0.000000
6        0.000000
7        0.000000
8        0.000000
9        0.000000
Name: 000060, dtype: float64

# 11월 29일 ~ 12월 3일 주식 종가 예측

In [16]:
model = lightgbm.LGBMRegressor(random_state=13)

days = 5
label_columns = ['after_1', 'after_2', 'after_3', 'after_4', 'after_5', ]
predict_days = 5 * -1 

accuracy = []
for code in tqdm(stock_list['종목코드'].values):
    data = fdr.DataReader(code, start = start_date, end = end_date).reset_index()
    data = feature_engineering(data)
    
    final_x = data.iloc[-1:]
    final_x = final_x.drop(['Date'] + label_columns, axis=1)
    # print(final_x);break
    x_val = data.iloc[-6:-5]
    x_val = x_val.drop(['Date'] + label_columns, axis=1)
    
    y_val = data.Close[predict_days:]

    # 이전 day 값이 없는 row 제외, 예측하는 날의 row 제외.
    train = data.drop(['Date'], axis=1)[days:predict_days]

    # after 1~5 columns 제외.
    x_train = train.drop(label_columns, axis=1)

    predictions = []
    final_predictions = []
    answers = []
    # 1일 후 부터 5일 후 까지 각각 예측.
    for j in range(1, 6):
        y_train = train[f'after_{j}']
        try:
            model.fit(x_train, y_train)
        except:
            print(f'code:{code}')

        output = model.predict(x_val)
        predictions.append(output[0])

        final_output = model.predict(final_x)
        final_predictions.append(final_output[0])
        
    accuracy.append(NMAE(predictions, y_val))
    sample_submission.loc[5:,code] = predictions
print(f'na:{sample_submission.isna().sum().sum()}')
print(f'acc:{sum(accuracy)/len(accuracy)}')

  0%|                                                                                                                                                                                                                 | 0/370 [00:00<?, ?it/s]/tmp/ipykernel_1457/1446661267.py:47: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'{col}_day_{lag}'] = -1
/tmp/ipykernel_1457/1446661267.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[day] = 0
/tmp/ipykernel_1457/1446661267.py:56: SettingWithCopyWarning: 
A value is trying to be

na:0
acc:0.005386868179120179


In [17]:
sample_submission['000060']

0    27888.279695
1    29232.221942
2    30262.127615
3    29487.604504
4    29560.781304
5    29344.299987
6    29613.547583
7    30672.441333
8    31203.071038
9    31597.973059
Name: 000060, dtype: float64

In [18]:
columns = list(sample_submission.columns[1:])

columns = ['Day'] + [str(x).zfill(6) for x in columns]

sample_submission.columns = columns

In [19]:
sample_submission.to_csv('last.csv',index=False)